## 安装依赖

如果当前环境已经有相关依赖了，则不用执行

In [ ]:
!pip install transformers[torch] datasets==3.6.0 evaluate

## 加载数据

In [ ]:
from datasets import load_dataset

# squad_v2等于True或者False分别代表使用SQUAD v1 或者 SQUAD v2。
# 如果您使用的是其他数据集，那么True代表的是：模型可以回答“不可回答”问题，也就是部分问题不给出答案，而False则代表所有问题必须回答。
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

datasets = load_dataset("squad_v2" if squad_v2 else "squad")

In [ ]:
# 你可以查看dataset对象都封装了什么东西
datasets

## 数据集可视化

为了能够进一步理解数据长什么样子，下面的函数将从数据集里随机选择几个例子进行展示。

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
  assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
  picks = []
  for _ in range(num_examples):
      pick = random.randint(0, len(dataset)-1)
      while pick in picks:
          pick = random.randint(0, len(dataset)-1)
      picks.append(pick)


  df = pd.DataFrame(dataset[picks])
  for column, typ in dataset.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])
    elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
        df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
  display(HTML(df.to_html()))

show_random_elements(datasets["train"], num_examples=2)

## 数据预处理

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer("What is your name?"))
print("单个文本tokenize: {}".format(tokenizer.tokenize("What is your name?")))
# add_special_tokens=True：会把[CLS][SEP]等特殊token拼接上去
print("单个文本tokenize: {}".format(tokenizer.tokenize("What is your name?",add_special_tokens=True)))
# [SEP]的id是102，会把两个句子的tokenids拼一起，然后用102分隔；
print(tokenizer("What is your name?", "My name is Sylvain."))

### 当question和context序列太长怎么处理？

定义两个变量，一个是最大长度，一个是重复元素个数；

超过最大长度自动截断，重复元素指一个长序列被拆分后，序列与序列之间的重复元素，避免序列被分隔后语义被破坏；

In [ ]:
# 特征的最大长度（问题和上下文）
max_length = 384

for i, example in enumerate(datasets["train"]):
  if len(tokenizer(example["question"],example["context"])["input_ids"]) > max_length:
    break

example = datasets["train"][i]

len(tokenizer(example["question"], example["context"])["input_ids"])

> 可以看到，如果不截断的话，长度是396

如果直接按max_length的限制截断，就会损失一些内容，导致语义被破坏；

In [ ]:
# 按比例切分后序列之间的重复长度
doc_stride = 128

tokenized_example = tokenizer(example["question"],example["context"],truncation="only_second",max_length=max_length,stride=doc_stride,return_overflowing_tokens=True)

print([len(x) for x in tokenized_example["input_ids"]])

> runcation="only_second"：表示指截断第二个序列（context）
>
> stride=doc_stride：表示按比例切分后序列之间的重复长度
>
> return_overflowing_tokens=True：表示当输入序列被截断（超过 max_length）时，返回所有溢出的 token（即被截断掉的部分），并生成多个“滑动窗口”形式的输入样本。

> 假设max_length=10, stride=4,上下文tokens为：
>
> [CLS] Q1 Q2 [SEP] C1 C2 C3 C4 C5 C6 C7 C8 C9 C10 C11 C12 [SEP]
>
> 就会被切成：
>
> 窗口1: [CLS] Q1 Q2 [SEP] C1 C2 C3 C4 C5 C6 [SEP]
>
> 窗口2: [CLS] Q1 Q2 [SEP] C5 C6 C7 C8 C9 C10 [SEP]
>
> 窗口3: [CLS] Q1 Q2 [SEP] C9 C10 C11 C12 [SEP]

### 新问题


由于序列被切分后，溢出部分和question序列组成了新的序列对；原本数据集中标注好的answer的start_id已经对不上了；怎么处理？

可以让tokenizer返回offsets_mapping，它可以返回每个token在切片后的位置和原始未切分时的位置的对应关系；

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
# 打印切片前后位置下标的对应关系
tokenized_example["offset_mapping"][0][:100]

> [(0, 0), (0, 3), (4, 8), (9, 13), (14, 18), (19, 22), ...., (409, 415), (416, 418)]
>
> 第一个token是[CLS],他不属于question和context，所以是(0,0)；
>
> 第二个token的起始和结束位置(0,3), 我们可以根据切片后的token id转化对应的token；然后使用offset_mapping参数映射回切片前的token位置，找到原始位置的tokens；
>
> 由于question拼接在context前面，所以直接从question里根据下标找就行了；

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
print(first_token_id) # 2129
offsets = tokenized_example["offset_mapping"][0][1]
print(offsets) # (0, 3)
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]]) # how How

这样就可以找到切分后的token和原始token的位置对应关系，接下来只需要处理原始token是在序列1还是序列2就行，通过sequence_ids()这个api就可以拿到每个token的序列下标；

In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

> [None, 0, 0,..., 0, None, 1, 1, 1, 1, ..., 1, 1, None]
>
> 这几个None就是[CLS]和[SEP]，因为他们不属于任何一个序列，只是用于占位，所以为None；

现在我们把上面的步骤整合起来，要找到标注样本的答案，在分割后的序列的位置；（新的位置）

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# 找第二个序列context的起始和结束位置
# 找到当前文本的start token index
token_start_index = 0
while sequence_ids[token_start_index] != 1:
  token_start_index += 1

# 找到当前文本的end token index
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# 检测答案是否在文本区间的外部，这种情况下意味着该样本的数据标注在CLS token位置。
offsets = tokenized_example["offset_mapping"][0]
if(offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
  # 将token_start_index和token_end_index移动到answer所在位置的两侧.
  # 注意：答案在最末尾的边界条件.
  while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
    token_start_index += 1
  start_position = token_start_index - 1
  while offsets[token_end_index][1] >= end_char:
    token_end_index -= 1
  end_position = token_end_index + 1
  print("start_position: {}, end_position: {}".format(start_position, end_position))
else:
  print("The answer is not in this feature.")

验证一下这个位置的token和标注样本中的answer是否一致：

In [ ]:
# 验证转换后的id对应的token跟答案是否准确
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

### 新问题

现在是固定了question是序列1，context是序列2；

那不同模型要求不同，可能要求相反的位置，怎么解决？

需要用多一个变量才处理一下:padding_side

In [ ]:
pad_on_right = tokenizer.padding_side == "right" #context在右边

## 封装完整的数据预处理函数

In [ ]:
def prepare_train_features(examples):
  # 既要对examples进行truncation（截断）和padding（补全）还要还要保留所有信息，所以要用的切片的方法。
  # 每一个超长文本example会被切片成多个输入，相邻两个输入之间会有交集。
  tokenized_examples = tokenizer(
      examples["question" if pad_on_right else "context"],
      examples["context" if pad_on_right else "question"],
      truncation="only_second" if pad_on_right else "only_first",
      max_length=max_length,
      stride=doc_stride,
      return_overflowing_tokens=True,
      return_offsets_mapping=True,
      padding="max_length", # 长度不足max_length的序列会被填充到max_length长度
  )
  # 我们使用overflow_to_sample_mapping参数来映射切片片ID到原始ID。
  # 比如有2个expamples被切成4片，那么对应是[0, 0, 1, 1]，前两片对应原来的第一个example。
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
  # offset_mapping也对应4片
  # offset_mapping参数帮助我们映射到原始输入，由于答案标注在原始输入上，所以有助于我们找到答案的起始和结束位置。
  offset_mapping = tokenized_examples.pop("offset_mapping")
  # 重新标注数据
  tokenized_examples["start_positions"] = []
  tokenized_examples["end_positions"] = []

  for i, offsets in enumerate(offset_mapping):
    # 对每一片进行处理，将无答案的样本标注到CLS上
    input_ids = tokenized_examples["input_ids"][i]
    cls_index = input_ids.index(tokenizer.cls_token_id)
    # 区分question和context
    sequence_ids = tokenized_examples.sequence_ids(i)
    # 拿到原始的example 下标.
    sample_index = sample_mapping[i]
    answers = examples["answers"][sample_index]
    # 如果没有答案，则使用CLS所在的位置为答案.
    if len(answers["answer_start"]) == 0:
      tokenized_examples["start_positions"].append(cls_index)
      tokenized_examples["end_positions"].append(cls_index)
    else:
      # 答案的character级别Start/end位置.
      start_char = answers["answer_start"][0]
      end_char = start_char + len(answers["text"][0])
      # 找到token级别的index start.
      token_start_index = 0
      while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
          token_start_index += 1
      # 找到token级别的index end.
      token_end_index = len(input_ids) - 1
      while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
          token_end_index -= 1
      # 检测答案是否超出文本长度，超出的话也用CLS index作为标注.
      if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
        tokenized_examples["start_positions"].append(cls_index)
        tokenized_examples["end_positions"].append(cls_index)
      else:
        # 如果不超出则找到答案token的start和end位置。.
        # Note: we could go after the last offset if the answer is the last word (edge case).
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
          token_start_index += 1
        tokenized_examples["start_positions"].append(token_start_index - 1)
        while offsets[token_end_index][1] >= end_char:
          token_end_index -= 1
        tokenized_examples["end_positions"].append(token_end_index + 1)

  return tokenized_examples


同样的，调用预处理函数后，返回的是一个字典结构的数据，模型无法直接使用，需要封装成dataset对象；

这里还需要remove_columns=datasets["train"].column_names，把数据集中的字段列表['id', 'context', 'question', 'answers']移除掉，只把特征映射回dataset即可；

**因为模型在计算loss的时候，不需要用到question，context等字段，留着占内存，所以删掉；**

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

## 加载预训练模型

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

## 创建训练参数配置类

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    f"test-squad",
    eval_strategy = "epoch",
    learning_rate=2e-5, #学习率
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, # 训练的论次
    weight_decay=0.01,
    report_to="none"  # 关闭wandb等
)

## 创建训练器

In [ ]:
from transformers import Trainer, default_data_collator

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

## 开始训练

In [ ]:
trainer.train()

In [ ]:
# 由于训练的时长太长，一个epoch要两个小时，所以保存一下权重
trainer.save_model("test-squad-trained") # 保存训练权重

## 模型评估（较复杂）

评估需要做一个处理，模型本身是预测的answer所在位置的概率，例如：
> 答案起始位置的概率：(tensor([ 46,  57,  78,  43, 118,  15,  72,  35,  15,  34,  73,  41,  80,  91,  156,  35], device='cuda:0')
>
> 答案结束位置的概率： tensor([ 47,  58,  81,  55, 118, 110,  75,  37, 110,  36,  76,  53,  83,  94,  158,  35], device='cuda:0'))

正常我们还需要考虑，找不到答案的情况，或者答案的位置下标不合法，或者答案的位置指向了question，这都是不行的；

这种情况一个比较好处理的就是，分数最高的位置如何不合法，那取第二高？第三高？

所以我们会用一个变量来限制取前n_best_size高的分数，过滤掉不合法的位置后，再排序，取第一个就可以了；

例如：
> [
> {'score': 16.706663, 'text': 'Denver Broncos'},
> {'score': 14.635585,'text': 'Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers'},
> {'score': 13.234194, 'text': 'Carolina Panthers'},
> {'score': 12.468662, 'text': 'Broncos'},
> {'score': 11.709289, 'text': 'Denver'}
> ]

如果要检查答案位置的文本是否在context里，需要对验证数据集的特征集进行处理；

一个样本example会对应多个feature，可能因为过长被切分成多个；

以及每个原始token的位置与切分后token的位置对应关系：offset_mapping;

## 验证数据集的预处理

In [ ]:
# 对验证集样本进行特征预处理
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # 移除掉overflow_to_sample_mapping字段
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # 添加一个字段
    tokenized_examples["example_id"] = []

    # 遍历每个样本里的每个特征集
    for i in range(len(tokenized_examples["input_ids"])):
        # 拿到序列id,是属于第一个序列还是第二个
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # 获取对应序列id对应的所有特征集
        sample_index = sample_mapping[i]
        # tokenized_examples["example_id"] = ['abc', 'abc', 'def'] 就是每个feature对应的原始样本ID
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # 将非 context 的 token offset 置为 None（比如 [CLS], question 部分），方便后处理时只关注 context 位置
        # 例如原始 offset_mapping:[(0,0), (0,5), (6,8), (9,12), (13,19), None, (0,3), (4,9)]
        # 清理后:[None, None, None, None, None, None, (0,3), (4,9)]  # 仅保留context
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names # 删掉原始的字段,不要占内存
)
# 获得所有预测结果 (start_logits, end_logits)
raw_predictions = trainer.predict(validation_features)

### 预处理函数的代码详解

In [ ]:
examples = validation_features
tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

处理后tokenized_examples的内部有三个属性，其数据样本为：

> tokenized_examples["input_ids"][0][:10]
>
> input_ids的输出：[101, 2073, 2003, 1996, 9440, 3305, 102, 1996, 9440, 3305]
>
> 特征对应的token为： [CLS] where is the eiffel tower ? [SEP] the eiffel tower...
>
> tokenized_examples["attention_mask"][0][:10]
>
> attention_mask的输出：[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
>
> tokenized_examples["offset_mapping"][0][:5]
>
> offset_mapping的输出为：[(0, 0), (0, 5), (6, 8), (9, 12), (13, 19)]   # token->原文字符区间

In [ ]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

因为长 context 会被分割成多个 feature，这里记录每个 feature 对应哪个原始样本。

`.pop` 会从 `tokenized_examples` 中移除该字段。

sample_mapping = [0, 0, 1]  表示前两个 feature 来自第0号样本，最后一个来自第1号样本。

## 验证数据集后置处理函数

In [ ]:
import collections
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, tokenizer, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    # 把原本的id list转成 kv形式方便定位每个example
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    # 用来存储每个 example 对应的 feature 索引（因为长文本会被切成多个特征）。
    features_per_example = collections.defaultdict(list)
    # 一个example可能对应多个特征集，所以要把多个特征集都归到同一个example下
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # 有序字典，用于存储每个样本的预测答案
    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} examples split into {len(features)} features.")

    for example_index in tqdm(range(len(examples))):
        example = examples[example_index]
        # 获取当前样本的特征集
        feature_indices = features_per_example[example_index]
        # 如果是 squad_v2，用于记录预测为 "无答案" 的得分（CLS token 分数）
        min_null_score = None
        # 保存当前样本的所有候选答案
        valid_answers = []
        context = example["context"]

        for feature_index in feature_indices:
            # 取出该 feature 对应的预测分数和 offset_mapping（token 到原文字符位置的映射）
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # 分词后的token位置和原始token的对应位置
            offset_mapping = features[feature_index]["offset_mapping"]

            # 找出 [CLS] 位置（模型用它表示无答案）
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            # 计算无答案得分（CLS 起点 + CLS 终点）
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            # 找当前样本所有切片中的最低无答案得分
            if min_null_score is None or feature_null_score < min_null_score:
                min_null_score = feature_null_score

            # 去分数最高的n_best_size个起点/终点，例如：
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

            # 这里是双层for遍历所有组合
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 超出上下文范围、offset 为 None、终点在起点前、答案长度超限的，都不要
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    # 记录合法的答案
                    valid_answers.append({
                        "score": start_logits[start_index] + end_logits[end_index],
                        "text": context[start_char:end_char],
                    })
        # 排序后取出最高分的答案
        best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0] if valid_answers else {"text": "", "score": 0.0}
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            predictions[example["id"]] = best_answer["text"] if best_answer["score"] > min_null_score else ""

    return predictions


final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

### 后置处理函数的代码详解

In [ ]:
all_start_logits, all_end_logits = raw_predictions

> 这里的raw_predictions是原始的预测logits:
>
> `all_start_logits[i][j]` 表示第 i 个特征的第 j 个 token 是起始词的概率分数
>
> 例如: all_start_logits[0][:5] 输出:[-3.1, 1.2, 0.8, -0.5, 2.3]

In [ ]:
example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

> 这是为了建立样本 ID 到索引的映射，方便快速定位每个 example;
>
> 假设:examples["id"] = ["abc", "def"]
>
> 那么example_id_to_index 就会输出: {"abc": 0, "def": 1}

In [ ]:
features_per_example = collections.defaultdict(list)

> 用来存储每个 example 对应的 feature 索引（因为长文本会被切成多个特征）。

In [ ]:
features = validation_features

for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

> 一个example可能对应多个特征集，所以要把多个特征集都归到同一个example下;
>
> 假设: 处理后 features_per_example = {0: [0, 1],  1: [2]}
>
> 说明 example[0] 对应 features[0] 和 features[1] ; example[1] 就对应 features[2]; 以此类推..

In [ ]:
# 这个是有序字典，用于存储每个样本的预测答案。
predictions = collections.OrderedDict()

In [ ]:
# 找出 [CLS] 位置（模型用它表示无答案）
cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
# 计算无答案得分（CLS 起点 + CLS 终点）
feature_null_score = start_logits[cls_index] + end_logits[cls_index]
# 找当前样本所有切片中的最低无答案得分
if min_null_score is None or feature_null_score < min_null_score:
    min_null_score = feature_null_score

> 这里主要需要计算以下最低的分数，当我们用的squad_v2的数据集，需要获取过滤掉小于最小分数的值；因为squad_v2的数据集是允许存在无答案的情况的；无答案一般是通过[CLS]位置的分数来作为概率；

In [ ]:
# 去分数最高的n_best_size个起点/终点
# 比如n_best_size为3：那么start_indexes = [3, 2, 5]
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()

# 这里是双层for遍历所有组合
for start_index in start_indexes:
    for end_index in end_indexes:
        # 超出上下文范围、offset 为 None、终点在起点前、答案长度超限的，都不要
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue

        start_char = offset_mapping[start_index][0]
        end_char = offset_mapping[end_index][1]
        # 记录合法的答案
        valid_answers.append({
            "score": start_logits[start_index] + end_logits[end_index],
            "text": context[start_char:end_char],
        })
# 排序后取出最高分的答案
best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0] if valid_answers else {"text": "", "score": 0.0}

> 这段代码主要是取原始n个最高分，然后过滤掉位置不合法的答案后，再做排序，取最大值；

## 完整代码，请看train.py